In [1]:
#import basic modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(color_codes=True)

In [2]:
train_activity = pd.read_csv('../lite_data/train_activity_lite.csv')
raw_train_activity = pd.read_csv('../transformed/real_values_int.csv')
raw_test_activity = pd.read_csv('../transformed/test_real_datas_int.csv')
time_first = pd.read_csv('../Features/First_week.csv')

In [3]:
#### 컬럼 순서 new_id / wk / 나머지....
cols = raw_train_activity.columns
raw_train_activity = raw_train_activity.loc[:,[cols[-1],cols[-2]] + cols[0:-2].tolist()].copy()
raw_test_activity = raw_test_activity.loc[:,[cols[-1],cols[-2]] + cols[0:-2].tolist()].copy()
cols = raw_train_activity.columns

In [4]:
#### 변수 정의 - 처음
## X_train, X_test, y_train, label_map
X_train = train_activity.groupby(by='new_id',sort=False,as_index=False).sum().iloc[:,:1]

In [7]:
#### error data
error_set = pd.read_csv('error_set_0.741.csv')

In [10]:
error_set.loc[error_set.new_id=='tr52531',:]

,new_id,label,y_pred
52531,tr52531,retained,2month


In [ ]:
'cnt_dt'
'play_time'
'game_combat_time'
'get_money'
'cnt_use_buffitem'
'party_chat'
'guild_chat'
---
고렙컨텐츠
저렙컨텐츠  증감...

In [11]:
raw_train_activity.columns.tolist()

['new_id',
 'wk',
 'payment_amount',
 'cnt_dt',
 'play_time',
 'npc_exp',
 'npc_hongmun',
 'quest_exp',
 'quest_hongmun',
 'item_hongmun',
 'game_combat_time',
 'get_money',
 'duel_cnt',
 'duel_win',
 'partybattle_cnt',
 'partybattle_win',
 'cnt_enter_inzone_solo',
 'cnt_enter_inzone_light',
 'cnt_enter_inzone_skilled',
 'cnt_enter_inzone_normal',
 'cnt_enter_raid',
 'cnt_enter_raid_light',
 'cnt_enter_bam',
 'cnt_clear_inzone_solo',
 'cnt_clear_inzone_light',
 'cnt_clear_inzone_skilled',
 'cnt_clear_inzone_normal',
 'cnt_clear_raid',
 'cnt_clear_raid_light',
 'cnt_clear_bam',
 'normal_chat',
 'whisper_chat',
 'district_chat',
 'party_chat',
 'guild_chat',
 'faction_chat',
 'cnt_use_buffitem',
 'gathering_cnt',
 'making_cnt']

In [5]:
train_time_var = pd.merge(time_first, raw_train_activity.loc[:,['new_id','wk','cnt_dt', 'play_time', 'game_combat_time', 'get_money', 'cnt_use_buffitem']])
train_time_var = train_time_var.loc[train_time_var.wk >= train_time_var.first_week,:]
train_time_var.drop('first_week',axis = 1,inplace = True)

In [6]:
#### 핵심 변수만!!!
time_cols = train_time_var.columns.tolist()
num_var = len(time_cols)

In [7]:
# 일반화
for i in range(2,num_var):
    ## pivoting
    train_time = train_time_var.iloc[:,[0,1,i]].pivot(index = 'new_id',columns = 'wk',values=time_cols[i])
    train_time = train_time.reindex(index = train_time.index.to_series().str.replace('tr','').astype(int).sort_values().index)# 인덱스 수정
    
    #### Feature Engineering

    ### time - series
    # fe_0 base statistics
    
    # median
    train_temp = pd.DataFrame(train_time.T.median(),columns=[time_cols[i] + '_median'])
    X_train = pd.merge(X_train,train_temp,how="left",left_on='new_id',right_on=train_temp.index)
    
    # mean
    train_temp = pd.DataFrame(train_time.T.mean(),columns=[time_cols[i] + '_mean'])
    X_train = pd.merge(X_train,train_temp,how="left",left_on='new_id',right_on=train_temp.index)
    
    # var(irregularity)
    train_temp = pd.DataFrame(train_time.T.var(),columns=[time_cols[i] + '_var']).fillna(0.0)
    X_train = pd.merge(X_train,train_temp,how="left",left_on='new_id',right_on=train_temp.index)
    
    # skew
    train_temp = pd.DataFrame(train_time.T.skew(),columns=[time_cols[i] + '_skew']).fillna(0.0)
    X_train = pd.merge(X_train,train_temp,how="left",left_on='new_id',right_on=train_temp.index)
    
    # kurt
    train_temp = pd.DataFrame(train_time.T.kurt(),columns=[time_cols[i] + '_kurt']).fillna(0.0)
    X_train = pd.merge(X_train,train_temp,how="left",left_on='new_id',right_on=train_temp.index)
    
    # fe_1 : 이동평균 MA(4)
    train_temp = train_time.rolling(window = 4,axis=1).mean().iloc[:,3:].copy()
    train_temp.columns = [time_cols[i] + x for x in ['_MA_1','_MA_2','_MA_3','_MA_4','_MA_5']]
    X_train = pd.merge(X_train,train_temp,how="left",left_on='new_id',right_on=train_temp.index)
    
    # fe_6 : autocorrelation
    train_temp = train_time.apply(lambda x: x.autocorr(), axis = 1).fillna(0)
    X_train = pd.merge(X_train,pd.DataFrame(train_temp,columns=[time_cols[i] +'_cycle']),how="left",left_on='new_id',right_on = train_temp.index)
    
    print(i)

/opt/conda/lib/python3.6/site-packages/numpy/lib/function_base.py:3175: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
/opt/conda/lib/python3.6/site-packages/numpy/lib/function_base.py:3109: RuntimeWarning: divide by zero encountered in double_scalars
  c *= 1. / np.float64(fact)


2
3
4
5
6


In [9]:
X_train.to_csv('X_train_real_play_time.csv',index = False)